# Movie Recommender System

This notebook builds a movie recommendation system using the MovieLens dataset and collaborative filtering.


# Business Understanding

The goal of this project is to build a **Movie Recommendation System** using the MovieLens dataset.  

### Problem Statement
Movie streaming platforms need effective ways to recommend movies that users are likely to enjoy.  
This project applies **collaborative filtering** to recommend movies based on past user ratings.

### Objectives
- Provide personalized movie recommendations.  
- Explore user-item interactions from the MovieLens dataset.  
- Build a baseline recommender using item-based collaborative filtering.  


# Data Understanding

The MovieLens dataset contains multiple CSV files:
- `movies.csv`: movieId, title, genres  
- `ratings.csv`: userId, movieId, rating, timestamp  
- `tags.csv`: userId, movieId, tag, timestamp  
- `links.csv`: movieId, IMDb ID, TMDb ID  

We will primarily use `ratings.csv` and `movies.csv` to build our recommender.


In [1]:
import pandas as pd

# Load data
movies = pd.read_csv("../data/movies.csv")
ratings = pd.read_csv("../data/ratings.csv")
tags = pd.read_csv("../data/tags.csv")
links = pd.read_csv("../data/links.csv")

# Inspect shapes
print("Movies:", movies.shape)
print("Ratings:", ratings.shape)
print("Tags:", tags.shape)
print("Links:", links.shape)

# Preview datasets
display(movies.head())
display(ratings.head())


Movies: (9742, 3)
Ratings: (100836, 4)
Tags: (3683, 4)
Links: (9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
